In [1]:
import os
import csv
import numpy as np
import matplotlib.pyplot as plt
import tools.mw_transmission_optimization as mto
from scipy.interpolate import interp1d
import datetime

# Launch NA qudi HW module with device in analyzer mode

In [3]:
FILENAME = datetime.datetime.now().strftime("%Y%m%d-%H%M-%S") + "_NetworkAnalysis.csv"
FILE_PATH = os.path.join(savelogic.data_dir, FILENAME)

print(f"Saving data to {FILE_PATH}")

Saving data to Z:\Data\20220112_SingleNV_Membrane_RT\20220113-1609-20_NetworkAnalysis.csv


# Get current transmission function
Invert it and interpolate, generate new data set with resolution of the SMF100A (within the memory constraints)


In [4]:
frequency, power = na_zvl6.get_trace(points=4000)
corrected_power = mto.get_corrected_power(power)

mto.write_to_csv(frequency, power, corrected_power, FILE_PATH)

In [5]:
FILE_PATH = "Z:\\Data\\20220112_SingleNV_Membrane_RT\\20220113-1549-14_NetworkAnalysis.csv"

In [6]:
frequency, power, corrected_power = mto.read_from_csv(FILE_PATH)

In [7]:
FREQUENCY_PRECISION = 500e3

f = interp1d(frequency, corrected_power)
frequency_interp = np.arange(1e9, 6e9, FREQUENCY_PRECISION)
corrected_power_interp = f(frequency_interp)

In [8]:
fig, ax = plt.subplots()

ax.plot(frequency, power, label="Original Transmission")
ax.plot(frequency, corrected_power, label="Corrected transmission")
ax.plot(frequency_interp, corrected_power_interp, label="Interpolated correction")
ax.plot(frequency, power + corrected_power, label="Expected outcome")
ax.set(xlabel="Frequency", ylabel="Power (dBm)")
ax.legend()
plt.savefig(FILE_PATH[:-4] + ".png", dpi=600)

# Send the interpolated values to the MW source
With the precision defined by the memory limit of the MW source (~500 kHz for 1 - 6 GHz range)

In [9]:
frequency_string, corrected_power_string = mto.generate_f_p_strings(frequency_interp, corrected_power_interp)

In [18]:
mw_smf100a.inst.timeout = 5000

In [19]:
mw_smf100a.disable_user_correction()
mw_smf100a.create_user_correction_file(name="odmr")
mw_smf100a.write_string_to_device(frequency_string, corrected_power_string)
print(mw_smf100a.check_number_of_points())

('10000\n', '10000\n')


In [20]:
mw_smf100a.load_user_correction_and_turn_on(name="odmr")